In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from datetime import datetime
from collections import defaultdict, Counter

%matplotlib inline

In [2]:
path_to_checkins = '../data/Gowalla/Gowalla_totalCheckins.txt'
checkins_df = pd.read_csv(path_to_checkins, sep='\t', names=['user_id', 'datetime', 'lat', 'lon', 'place_id'])

In [3]:
checkins_df.head()

,user_id,datetime,lat,lon,place_id
0,0,2010-10-19T23:55:27Z,30.235909,-97.795140,22847
1,0,2010-10-18T22:17:43Z,30.269103,-97.749395,420315
2,0,2010-10-17T23:42:03Z,30.255731,-97.763386,316637
3,0,2010-10-17T19:26:05Z,30.263418,-97.757597,16516
4,0,2010-10-16T18:50:42Z,30.274292,-97.740523,5535878


In [4]:
checkins_df['timestamp'] = checkins_df.apply(
    lambda row: int(datetime.strptime(row['datetime'], "%Y-%m-%dT%H:%M:%SZ").timestamp()),
    axis=1
)

checkins_df = checkins_df.drop(['datetime'], axis=1)
checkins_df.head()

,user_id,lat,lon,place_id,timestamp
0,0,30.235909,-97.795140,22847,1287525327
1,0,30.269103,-97.749395,420315,1287433063
2,0,30.255731,-97.763386,316637,1287351723
3,0,30.263418,-97.757597,16516,1287336365
4,0,30.274292,-97.740523,5535878,1287247842


In [5]:
checkins_df.isnull().sum()

user_id      0
lat          0
lon          0
place_id     0
timestamp    0
dtype: int64

In [6]:
checkins_df.user_id.max(), checkins_df.user_id.unique().shape

(196585, (107092,))

In [7]:
checkins_df.user_id = pd.factorize(checkins_df.user_id)[0] + 1
checkins_df.user_id.min(), checkins_df.user_id.max(), checkins_df.user_id.unique().shape

(1, 107092, (107092,))

In [8]:
checkins_df.place_id = pd.factorize(checkins_df.place_id)[0] + 1
checkins_df.place_id.min(), checkins_df.place_id.max(), checkins_df.place_id.unique().shape

(1, 1280969, (1280969,))

In [9]:
checkins_df.head()

,user_id,lat,lon,place_id,timestamp
0,1,30.235909,-97.795140,1,1287525327
1,1,30.269103,-97.749395,2,1287433063
2,1,30.255731,-97.763386,3,1287351723
3,1,30.263418,-97.757597,4,1287336365
4,1,30.274292,-97.740523,5,1287247842


In [11]:
sorted_timestamps = sorted(checkins_df.timestamp)
len(sorted_timestamps)

6442892

In [12]:
threshold_timestamp = sorted_timestamps[int(len(sorted_timestamps) * (1.0 - 0.1))]
threshold_timestamp

1286110393

In [13]:
train_data = checkins_df[checkins_df.timestamp < threshold_timestamp]
test_data = checkins_df[checkins_df.timestamp >= threshold_timestamp]

train_data.shape, test_data.shape

((5798600, 5), (644292, 5))

In [14]:
data = []

for _, row in tqdm(train_data.iterrows()):
    data.append({
        'user_id': int(row.user_id),
        'item_id': int(row.place_id),
        'timestamp': int(row.timestamp)
    })

print(len(data))

5798600it [04:32, 21286.77it/s]

5798600


In [15]:
user_history = defaultdict(list)
item_history = defaultdict(list)

for row in tqdm(data):
    user_raw_id = row['user_id']
    item_raw_id = row['item_id']
    interaction_timestamp = row['timestamp']
    
    user_history[user_raw_id].append({'item_id': item_raw_id, 'timestamp': interaction_timestamp})
    item_history[item_raw_id].append({'user_id': user_raw_id, 'timestamp': interaction_timestamp})


is_changed = True
threshold = 20
good_users = set()
good_items = set()


while is_changed:
    old_state = (len(good_users), len(good_items))
    
    good_users = set()
    good_items = set()

    for user_id, history in user_history.items():
        if len(history) >= threshold:
            good_users.add(user_id)

    for item_id, history in item_history.items():
        if len(history) >= threshold:
            good_items.add(item_id)
    
    user_history = {
        user_id: list(filter(lambda x: x['item_id'] in good_items, history))
        for user_id, history in user_history.items()
    }
    
    item_history = {
        item_id: list(filter(lambda x: x['user_id'] in good_users, history))
        for item_id, history in item_history.items()
    }
    
    new_state = (len(good_users), len(good_items))
    is_changed = (old_state != new_state)
    print(old_state, new_state)

100%|████████████████████████████████████████████████████████████████████| 5798600/5798600 [00:35<00:00, 165460.15it/s]


(0, 0) (53462, 43891)
(53462, 43891) (23158, 41282)
(23158, 41282) (22570, 34086)
(22570, 34086) (20839, 33812)
(20839, 33812) (20769, 32897)
(20769, 32897) (20530, 32867)
(20530, 32867) (20519, 32739)
(20519, 32739) (20482, 32732)
(20482, 32732) (20480, 32708)
(20480, 32708) (20471, 32706)
(20471, 32706) (20471, 32701)
(20471, 32701) (20471, 32701)


In [16]:
user_mapping = {}
item_mapping = {}
tmp_user_history = defaultdict(list)
tmp_item_history = defaultdict(list)

for user_id, history in tqdm(user_history.items()):
    processed_history = []

    for filtered_item in history:
        item_id = filtered_item['item_id']
        item_timestamp = filtered_item['timestamp']

        processed_item_id = item_mapping.get(item_id, len(item_mapping) + 1)
        item_mapping[item_id] = processed_item_id

        processed_history.append({'item_id': processed_item_id, 'timestamp': item_timestamp})
        
    if len(processed_history) >= threshold:
        processed_user_id = user_mapping.get(user_id, len(user_mapping) + 1)
        user_mapping[user_id] = processed_user_id

        tmp_user_history[processed_user_id] = sorted(processed_history, key=lambda x: x['timestamp'])

    
for item_id, history in tqdm(item_history.items()):
    processed_history = []

    for filtered_user in history:
        user_id = filtered_user['user_id']
        user_timestamp = filtered_user['timestamp']

        processed_user_id = user_mapping.get(user_id, len(user_mapping) + 1)
        user_mapping[user_id] = processed_user_id

        processed_history.append({'user_id': processed_user_id, 'timestamp': user_timestamp})

    if len(processed_history) >= threshold:
        processed_item_id = item_mapping.get(item_id, len(item_mapping) + 1)
        item_mapping[item_id] = processed_item_id

        tmp_item_history[processed_item_id] = sorted(processed_history, key=lambda x: x['timestamp'])

user_history = tmp_user_history
item_history = tmp_item_history

100%|████████████████████████████████████████████████████████████████████| 1191771/1191771 [00:04<00:00, 239835.76it/s]


In [17]:
print('Users count:', len(user_mapping))
print('Items count:', len(item_mapping))
print('Actions count:', sum(list(map(lambda x: len(x), user_history.values()))))
print('Avg user history len:', np.mean(list(map(lambda x: len(x), user_history.values()))))
print('Avg item history len:', np.mean(list(map(lambda x: len(x), item_history.values()))))

Users count: 20471
Items count: 32701
Actions count: 1597715
Avg user history len: 78.04772605148747
Avg item history len: 48.85829179535794


In [18]:
test_data_filtered = test_data[test_data.user_id.isin(user_mapping) & test_data.place_id.isin(item_mapping)]
test_data_filtered.shape, test_data.shape

((91465, 5), (644292, 5))

In [21]:
test_list = []

for _, row in tqdm(test_data_filtered.iterrows()):
    test_list.append({
        'user_id': int(row.user_id),
        'item_id': int(row.place_id),
        'timestamp': int(row.timestamp)
    })

print(len(test_list))

tmp_user_history = {}
for user_id, history in user_history.items():
    tmp_user_history[user_id] = sorted(history, key=lambda x: x['timestamp'])
user_history = tmp_user_history

test_list = sorted(test_list, key=lambda x: x['timestamp'])

test_user_history = defaultdict(list)

for row in tqdm(test_list):
    user_raw_id = row['user_id']
    item_raw_id = row['item_id']
    interaction_timestamp = row['timestamp']
    
    test_user_history[user_raw_id].append({'item_id': item_raw_id, 'timestamp': interaction_timestamp})
    
tmp_user_history = defaultdict(list)


for user_id, history in tqdm(test_user_history.items()):
    processed_history = []

    for filtered_item in history:
        item_id = filtered_item['item_id']
        item_timestamp = filtered_item['timestamp']

        processed_item_id = item_mapping[item_id]
        processed_history.append({'item_id': processed_item_id, 'timestamp': item_timestamp})
        
    processed_user_id = user_mapping[user_id]
    tmp_user_history[processed_user_id] = sorted(processed_history, key=lambda x: x['timestamp'])

    
test_user_history = tmp_user_history

91465it [00:04, 18372.18it/s]


91465


100%|█████████████████████████████████████████████████████████████████████████| 10963/10963 [00:00<00:00, 66483.08it/s]


In [22]:
with open('../data/Gowalla/train_new.txt', 'w') as f:
    for user_id, item_history in user_history.items():
        f.write(' '.join([str(user_id)] + [
            str(item_event['item_id']) for item_event in sorted(item_history, key=lambda x: x['timestamp'])
        ]))
        f.write('\n')

In [23]:
with open('../data/Gowalla/test_new.txt', 'w') as f:
    for user_id, item_history in test_user_history.items():
        f.write(' '.join([str(user_id)] + [
            str(item_event['item_id']) for item_event in sorted(item_history, key=lambda x: x['timestamp'])
        ]))
        f.write('\n')

## All data

In [26]:
data = []

for _, row in tqdm(checkins_df.iterrows()):
    data.append({
        'user_id': int(row.user_id),
        'item_id': int(row.place_id),
        'timestamp': int(row.timestamp)
    })

print(len(data))

6442892it [05:49, 18441.06it/s]

6442892


In [27]:
user_history = defaultdict(list)
item_history = defaultdict(list)

for row in tqdm(data):
    user_raw_id = row['user_id']
    item_raw_id = row['item_id']
    interaction_timestamp = row['timestamp']
    
    user_history[user_raw_id].append({'item_id': item_raw_id, 'timestamp': interaction_timestamp})
    item_history[item_raw_id].append({'user_id': user_raw_id, 'timestamp': interaction_timestamp})


is_changed = True
threshold = 20
good_users = set()
good_items = set()


while is_changed:
    old_state = (len(good_users), len(good_items))
    
    good_users = set()
    good_items = set()

    for user_id, history in user_history.items():
        if len(history) >= threshold:
            good_users.add(user_id)

    for item_id, history in item_history.items():
        if len(history) >= threshold:
            good_items.add(item_id)
    
    user_history = {
        user_id: list(filter(lambda x: x['item_id'] in good_items, history))
        for user_id, history in user_history.items()
    }
    
    item_history = {
        item_id: list(filter(lambda x: x['user_id'] in good_users, history))
        for item_id, history in item_history.items()
    }
    
    new_state = (len(good_users), len(good_items))
    is_changed = (old_state != new_state)
    print(old_state, new_state)

100%|█████████████████████████████████████████████████████████████████████| 6442892/6442892 [01:21<00:00, 79494.54it/s]


(0, 0) (60562, 49750)
(60562, 49750) (26336, 47198)
(26336, 47198) (25811, 38979)
(25811, 38979) (23856, 38730)
(23856, 38730) (23797, 37646)
(23797, 37646) (23509, 37610)
(23509, 37610) (23499, 37452)
(23499, 37452) (23443, 37451)
(23443, 37451) (23442, 37414)
(23442, 37414) (23431, 37414)
(23431, 37414) (23431, 37404)
(23431, 37404) (23429, 37404)
(23429, 37404) (23429, 37402)
(23429, 37402) (23427, 37402)
(23427, 37402) (23427, 37398)
(23427, 37398) (23427, 37398)


In [28]:
user_mapping = {}
item_mapping = {}
tmp_user_history = defaultdict(list)
tmp_item_history = defaultdict(list)

for user_id, history in tqdm(user_history.items()):
    processed_history = []

    for filtered_item in history:
        item_id = filtered_item['item_id']
        item_timestamp = filtered_item['timestamp']

        processed_item_id = item_mapping.get(item_id, len(item_mapping) + 1)
        item_mapping[item_id] = processed_item_id

        processed_history.append({'item_id': processed_item_id, 'timestamp': item_timestamp})
        
    if len(processed_history) >= threshold:
        processed_user_id = user_mapping.get(user_id, len(user_mapping) + 1)
        user_mapping[user_id] = processed_user_id

        tmp_user_history[processed_user_id] = sorted(processed_history, key=lambda x: x['timestamp'])

    
for item_id, history in tqdm(item_history.items()):
    processed_history = []

    for filtered_user in history:
        user_id = filtered_user['user_id']
        user_timestamp = filtered_user['timestamp']

        processed_user_id = user_mapping.get(user_id, len(user_mapping) + 1)
        user_mapping[user_id] = processed_user_id

        processed_history.append({'user_id': processed_user_id, 'timestamp': user_timestamp})

    if len(processed_history) >= threshold:
        processed_item_id = item_mapping.get(item_id, len(item_mapping) + 1)
        item_mapping[item_id] = processed_item_id

        tmp_item_history[processed_item_id] = sorted(processed_history, key=lambda x: x['timestamp'])

user_history = tmp_user_history
item_history = tmp_item_history

100%|████████████████████████████████████████████████████████████████████| 1280969/1280969 [00:06<00:00, 183525.04it/s]


In [29]:
print('Users count:', len(user_mapping))
print('Items count:', len(item_mapping))
print('Actions count:', sum(list(map(lambda x: len(x), user_history.values()))))
print('Avg user history len:', np.mean(list(map(lambda x: len(x), user_history.values()))))
print('Avg item history len:', np.mean(list(map(lambda x: len(x), item_history.values()))))

Users count: 23427
Items count: 37398
Actions count: 1849431
Avg user history len: 78.94442310154949
Avg item history len: 49.452671265843094


In [30]:
with open('../data/Gowalla/all_data.txt', 'w') as f:
    for user_id, item_history in user_history.items():
        f.write(' '.join([str(user_id)] + [
            str(item_event['item_id']) for item_event in sorted(item_history, key=lambda x: x['timestamp'])
        ]))
        f.write('\n')